In [1]:
#!pip install celery

In [2]:
#!sudo rabbitmqctl add_user barnabas barnabaspwd
#!sudo rabbitmqctl set_permissions -p / barnabas "." "." ".*"

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
barnabas	[]
guest	[administrator]
myguest	[]
myguest2	[]


on worker machine 1 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=3

on worker machine 2 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=4


In [4]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import json

import celery
from test_workers import message_to_workers1, message_to_workers2

import math
import socket


In [5]:
celery.current_app.control.inspect().ping()

{'celery@ip-172-31-3-73': {'ok': 'pong'},
 'celery@ip-172-31-0-150': {'ok': 'pong'}}

In [6]:
mykeys=list(celery.current_app.control.inspect().stats().keys())
print(mykeys)

['celery@ip-172-31-0-150', 'celery@ip-172-31-3-73']


In [7]:
socket.gethostbyname(socket.gethostname())

'172.31.0.150'

In [8]:
mydict=celery.current_app.control.inspect().stats()[mykeys[0]]
mydict['pool']

{'max-concurrency': 3,
 'processes': [21551, 21552, 21553],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 3, 'active': 0}}}

In [9]:
mydict=celery.current_app.control.inspect().stats()[mykeys[1]]
mydict['pool']

{'max-concurrency': 4,
 'processes': [19604, 19605, 19606, 19609],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 4, 'active': 0}}}

In [10]:
for iter in range(0,30):
    myresult1=message_to_workers1.delay(str(1000+iter)) 
    result1=myresult1.get()
    
    myresult2=message_to_workers2.delay(str(1000+iter))
    result2=myresult2.get()
    print('***')
    print(result1)
    print(result2)
    print('***')

***
['172.31.3.73_ForkPoolWorker-2__1000_140_88554****']
['172.31.0.150_ForkPoolWorker-2__1000_503_88553****']
***
***
['172.31.3.73_ForkPoolWorker-1__1001_140_88560****']
['172.31.0.150_ForkPoolWorker-3__1001_503_88558****']
***
***
['172.31.3.73_ForkPoolWorker-2__1002_204_88565****', '172.31.3.73_ForkPoolWorker-2__1000_140_88554****']
['172.31.0.150_ForkPoolWorker-1__1002_503_88563****']
***
***
['172.31.3.73_ForkPoolWorker-1__1003_204_88569****', '172.31.3.73_ForkPoolWorker-1__1001_140_88560****']
['172.31.0.150_ForkPoolWorker-2__1003_236_88566****', '172.31.0.150_ForkPoolWorker-2__1000_503_88553****']
***
***
['172.31.3.73_ForkPoolWorker-2__1004_121_88572****', '172.31.3.73_ForkPoolWorker-2__1002_204_88565****']
['172.31.0.150_ForkPoolWorker-3__1004_236_88570****', '172.31.0.150_ForkPoolWorker-3__1001_503_88558****']
***
***
['172.31.3.73_ForkPoolWorker-1__1005_121_88576****', '172.31.3.73_ForkPoolWorker-1__1003_204_88569****']
['172.31.0.150_ForkPoolWorker-1__1005_236_88574****', 